### Example how socket data is transformed from JSON to dictionary to dataframe to numpy array

In [1]:
import pandas as pd
import json
import timeit
import time
import numpy as np

In [2]:
drone_last_action = time.time()
time.sleep(1.2)
drone_status = 'grounded'

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
cols_x = ['leftShoulder_x',
          'rightShoulder_x',
          'leftWrist_x',
          'rightWrist_x',
          'leftHip_x',
          'rightHip_x',
          'leftElbow_x',
          'rightElbow_x']

cols_y = [col.replace('x', 'y') for col in cols_x]
cols = sorted(cols_x + cols_y)

In [5]:
pose_json = '{"poses":[{"score":0.3305602508857298,"keypoints":[{"score":0.9985909461975098,"part":"nose","position":{"x":248.6576838258837,"y":256.0402913411458}},{"score":0.9997169375419617,"part":"leftEye","position":{"x":288.0274458087859,"y":236.07972819010416}},{"score":0.9891241788864136,"part":"rightEye","position":{"x":218.41943759605533,"y":244.81049397786458}},{"score":0.9666750431060791,"part":"leftEar","position":{"x":335.9838671137075,"y":290.9280598958333}},{"score":0.786156952381134,"part":"rightEar","position":{"x":183.74859819255892,"y":301.9474772135417}},{"score":0.657204270362854,"part":"leftShoulder","position":{"x":443.04252449410865,"y":472.5755121866862}},{"score":0.16853265464305878,"part":"rightShoulder","position":{"x":119.14107726050206,"y":453.5830586751302}},{"score":0.016177650541067123,"part":"leftElbow","position":{"x":493.0340886350538,"y":489.2761617024739}},{"score":0.008160887286067009,"part":"rightElbow","position":{"x":-15.11167832671619,"y":497.9180094401042}},{"score":0.004399956203997135,"part":"leftWrist","position":{"x":466.03887689309045,"y":504.0832194010417}},{"score":0.004726673010736704,"part":"rightWrist","position":{"x":43.51104736328125,"y":498.2728698730469}},{"score":0.004330772906541824,"part":"leftHip","position":{"x":397.88377405385506,"y":502.5487772623698}},{"score":0.0069005475379526615,"part":"rightHip","position":{"x":91.98278208247952,"y":488.4844584147135}},{"score":0.003729963442310691,"part":"leftKnee","position":{"x":444.6972150958953,"y":471.8383005777995}},{"score":0.0024850412737578154,"part":"rightKnee","position":{"x":7.370285284323771,"y":500.6000691731771}},{"score":0.0016965661197900772,"part":"leftAnkle","position":{"x":444.20433669793806,"y":484.8676361083984}},{"score":0.0009152236161753535,"part":"rightAnkle","position":{"x":25.47802734375,"y":479.93550135294595}}]}],"image":{"width":640,"height":480}}'

In [6]:
pose_empty_json = '{"poses": [], "image": {"width": 640, "height": 480}}'

In [7]:
df = pd.DataFrame(columns=cols)

In [8]:
def json_to_dict(pose_json):

    x = json.loads(pose_json)
    pose_dict = {}
    
    if len(x['poses']) == 1:
        for i in range(8):
            pose_dict[x['poses'][0]['keypoints'][i+5]['part'] +
                      '_x'] = x['poses'][0]['keypoints'][i + 5]['position']['x']
            pose_dict[x['poses'][0]['keypoints'][i+5]['part'] +
                      '_y'] = x['poses'][0]['keypoints'][i + 5]['position']['y']

    return pose_dict

In [9]:
def steer_drone(movement):
    global drone_last_action
    global drone_status
    if (time.time() - drone_last_action) > 1:
        drone_last_action = time.time()
        if ((movement == 4) & (drone_status == 'grounded')):
            drone_status = 'flying'
            print('drone.takeoff()')
        if (drone_status != 'grounded'):
            if movement == 6:
                drone_status = 'grounded'
                print('drone.land()')
            if movement == 0:
                print("drone.flip('r')")
            if movement == 1:
                print('drone.rotate_ccw(45)')
            if movement == 2:
                print('drone.rotate_cw(45)')
            if movement == 5:
                print('drone.move_forward(1)')

In [10]:
json_to_dict(pose_json)

{'leftShoulder_x': 443.04252449410865,
 'leftShoulder_y': 472.5755121866862,
 'rightShoulder_x': 119.14107726050206,
 'rightShoulder_y': 453.5830586751302,
 'leftElbow_x': 493.0340886350538,
 'leftElbow_y': 489.2761617024739,
 'rightElbow_x': -15.11167832671619,
 'rightElbow_y': 497.9180094401042,
 'leftWrist_x': 466.03887689309045,
 'leftWrist_y': 504.0832194010417,
 'rightWrist_x': 43.51104736328125,
 'rightWrist_y': 498.2728698730469,
 'leftHip_x': 397.88377405385506,
 'leftHip_y': 502.5487772623698,
 'rightHip_x': 91.98278208247952,
 'rightHip_y': 488.4844584147135}

In [11]:
df = df.append(pd.DataFrame(json_to_dict(pose_json), index=[0]))
# df.to_csv('dataframe.csv',  index=False)
df.reset_index(inplace=True, drop=True)
df

,leftElbow_x,leftElbow_y,leftHip_x,leftHip_y,leftShoulder_x,leftShoulder_y,leftWrist_x,leftWrist_y,rightElbow_x,rightElbow_y,rightHip_x,rightHip_y,rightShoulder_x,rightShoulder_y,rightWrist_x,rightWrist_y
0,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287


In [12]:
for i in range(17):
    df = df.append(pd.DataFrame(json_to_dict(pose_json), index=[0]))
df

,leftElbow_x,leftElbow_y,leftHip_x,leftHip_y,leftShoulder_x,leftShoulder_y,leftWrist_x,leftWrist_y,rightElbow_x,rightElbow_y,rightHip_x,rightHip_y,rightShoulder_x,rightShoulder_y,rightWrist_x,rightWrist_y
0,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
0,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
0,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
0,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
0,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
0,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
0,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
0,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
0,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287
0,493.034089,489.276162,397.883774,502.548777,443.042524,472.575512,466.038877,504.083219,-15.111678,497.918009,91.982782,488.484458,119.141077,453.583059,43.511047,498.27287


In [13]:
pose_np = df.values.reshape(1, 18, 16)
pose_np[0][0]

array([493.03408864, 489.2761617 , 397.88377405, 502.54877726,
       443.04252449, 472.57551219, 466.03887689, 504.0832194 ,
       -15.11167833, 497.91800944,  91.98278208, 488.48445841,
       119.14107726, 453.58305868,  43.51104736, 498.27286987])

In [14]:
import os
os.getcwd()

'C:\\Users\\Christian\\git\\drone_steering\\models\\supporting_notebooks'

In [15]:
os.chdir('../..')

In [16]:
from app.module import GestureTransformer
from sklearn.pipeline import make_pipeline

In [17]:
processing_pipeline = make_pipeline(GestureTransformer(cols))
processing_pipeline.fit_transform(pose_np)[0][0]

array([-7.09125973, -0.307615  , -4.15790155, -0.71679216, -5.55008602,
        0.20724391, -6.25903311, -0.76409698,  8.5742013 , -0.57403172,
        5.27262092, -0.28320784,  4.43536664,  0.79275609,  6.76694031,
       -0.5849716 ])

In [18]:
from keras.models import load_model
import h5py

Using TensorFlow backend.


In [19]:
model = load_model('models/model_gesture.h5')
print(np.argmax(model.predict(pose_np)[0]))

0
